<a href="https://colab.research.google.com/github/Nikhitha-S-Pavan/Deep-learning-examples-using-keras/blob/main/Hyperparameter_reg_example_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#!pip install scikit-learn==0.21.2

In [ ]:
link = "https://drive.google.com/file/d/1xcQiEX1nRa_F-OyiHj6fnC_CGrO33ldQ/view?usp=sharing"

In [3]:
id= link.split('/')[-2]
print (id) # Verify that you have everything after '='

1xcQiEX1nRa_F-OyiHj6fnC_CGrO33ldQ


In [4]:
import pandas as pd

In [5]:
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Churn_Modelling.csv')  


In [6]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug  5 13:30:38 2019
@author: nikhitha pavan
"""

# Artificial Neural Network


# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
# Importing the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13]
y = dataset.iloc[:, 13]
print(dataset)

      RowNumber  CustomerId    Surname  ...  IsActiveMember EstimatedSalary Exited
0             1    15634602   Hargrave  ...               1       101348.88      1
1             2    15647311       Hill  ...               1       112542.58      0
2             3    15619304       Onio  ...               0       113931.57      1
3             4    15701354       Boni  ...               0        93826.63      0
4             5    15737888   Mitchell  ...               1        79084.10      0
...         ...         ...        ...  ...             ...             ...    ...
9995       9996    15606229   Obijiaku  ...               0        96270.64      0
9996       9997    15569892  Johnstone  ...               1       101699.77      0
9997       9998    15584532        Liu  ...               1        42085.58      1
9998       9999    15682355  Sabbatini  ...               0        92888.52      1
9999      10000    15628319     Walker  ...               0        38190.78      0

[10

In [8]:
#Create dummy variables
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X['Gender'],drop_first=True)

In [9]:
## Concatenate the Data Frames

X=pd.concat([X,geography,gender],axis=1)

print(X)

      CreditScore Geography  Gender  Age  ...  EstimatedSalary  Germany  Spain  Male
0             619    France  Female   42  ...        101348.88        0      0     0
1             608     Spain  Female   41  ...        112542.58        0      1     0
2             502    France  Female   42  ...        113931.57        0      0     0
3             699    France  Female   39  ...         93826.63        0      0     0
4             850     Spain  Female   43  ...         79084.10        0      1     0
...           ...       ...     ...  ...  ...              ...      ...    ...   ...
9995          771    France    Male   39  ...         96270.64        0      0     1
9996          516    France    Male   35  ...        101699.77        0      0     1
9997          709    France  Female   36  ...         42085.58        0      0     0
9998          772   Germany    Male   42  ...         92888.52        1      0     1
9999          792    France  Female   28  ...         38190.78   

In [10]:
## Drop Unnecessary columns
X=X.drop(['Geography','Gender'],axis=1)


In [11]:

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)


In [12]:

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [13]:

## Perform Hyperparameter Optimization
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid




In [14]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
            
    model.add(Dense(units = 1, kernel_initializer= 'glorot_uniform', activation = 'sigmoid')) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=0)




In [19]:
layers = [[20], [15,30],[15,30,45]]
activations = [ 'sigmoid','relu']

In [21]:
param_grid = dict(layers=layers, activation=activations, batch_size = [2], epochs=[5])
grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

grid_result = grid.fit(X_train, y_train)

[grid_result.best_score_,grid_result.best_params_]

[0.8538750052452088,
 {'activation': 'relu', 'batch_size': 2, 'epochs': 5, 'layers': [20]}]